In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
       
from HPIB.HP4155 import HP4155
from HPIB.HPT import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P
from HPIB.DevParams import UMC
from HPIB.INOSerial import Arduino

from BFmodule import DR

from YFunc import YFuncExtraction

def WriteLog(msg, path, mode='a', end='\n', output=True):
    with open(path, mode) as logfile:
        logfile.write(msg+end)
    if output:
        print(f"{msg}{end}", end='')

from IPython.display import clear_output, display
from os import makedirs, rename
from time import sleep
from datetime import datetime, timedelta

HP=HP4155("GPIB0::17", debug=False)
HP.SetIntTime("LONG")

INO=Arduino("COM3")
print(INO.ask('*'.encode()))

Elsa=DR()

HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
InoMatrix

Elsa v2.4.2


In [9]:
def TestDevice(device, chn, path, params):
    global HP, INO, Elsa, prog_bar
    print(INO.opench(chn+1))

    if not device:
        print(INO.opench(0))
        sleep(2)
        return 0

    if device[:2] not in ['CA', 'CB', 'TP', 'TN', 'DP', 'DN']:
        return "Invalid device"

    WriteLog(f"## Ch {chn+1} {device}", path + 'log.txt')
    pathp=path+device
    makedirs(pathp, exist_ok=True)
    
    ####################### Measure Diode
    #
    #
    if 'd' in device.lower():
        
        HP.SetStop("COMP")
        HP.SetIntTime("MED")
        HP.SingleDiode(params['Vfmin'], params['Vfmax'], params['Vfstep'], 'SMU2', 'SMU1', Comp=params['IComp'])
            
        now=datetime.now().strftime('%y%m%d %H%M%S')
        Plot(HP.SingleSave(f"{pathp}/{now}.csv", timeout=30, real=True), 'Vf', 'If')
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(f"{pathp}/{now}.csv", f"{pathp}/{temp} - {now}.csv")
        rename(f"{pathp}/{now}.png", f"{pathp}/{temp} - {now}.png")
        
        now=datetime.now()

        # with open(f"{pathp}/V100uA.log", 'a') as DiodeParam:
        #     DiodeParam.write(f"{temp},{format(RCB, '.2e')}\n")
            
        while True:
            if (datetime.now()-now).seconds > params['min_wait']: break
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
            sleep(0.5)
            
        HP.SetStop("OFF")
        HP.SetIntTime("LONG")
        
        print(INO.opench(0))
        WriteLog('', path + 'log.txt')
        return 0
    #
    #
    #######################

    ####################### Measure Transistor
    #
    #

    if 't' in device.lower():
        
        ptype='p' in device.lower()
        
        HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vd'], ptype=ptype)
  
        now=datetime.now().strftime('%y%m%d %H%M')
        LIN = PlotVgs(HP.SingleSave(f"{pathp}/IdVgs - {now}.csv", timeout=30))
        temp=format(Elsa.GetT('t4k'), '07.3f')
        newname=f"{pathp}/IdVgs - {temp} - {now}"
        rename(f"{pathp}/IdVgs - {now}.csv", f"{newname}.csv")
        rename(f"{pathp}/IdVgs - {now}.png", f"{newname}.png")
        now=datetime.now()
            
        WriteLog(f"Vth={LIN} V", path + 'log.txt')
        try:
            DGM=SecDer(f"{newname}.csv")
            WriteLog(f"SecDer={format(DGM, '.2f')} V", path + 'log.txt')
        except:
            WriteLog(f"SecDer Fail", path + 'log.txt')

        try:
            LIN, Vth, SS, migm, miyf, theta1, theta2, errmax = YFuncExtraction(f"{newname}.csv", UMC[int(device[2:])], 4.2, 3.9, params['Vd'])
            WriteLog(f"LIN={format(LIN, '.2f')}, Vth={format(Vth, '.2f')} V, SS={format(SS, '.1f')} mV/dec, miyf={format(migm, '.1f')}, miyf={format(miyf, '.1f')}, theta1={format(theta1, '.3e')}, theta2={format(theta2, '.3e')}", path + 'log.txt')
            WriteLog(f"{temp},{format(LIN, '.2f')},{format(Vth, '.2f')},{format(SS, '.1f')},{format(migm, '.1f')},{format(miyf, '.1f')},{format(theta1, '.3e')},{format(theta2, '.3e')}", pathp + '/params.txt')
        except:
            WriteLog(f"YFunc Fail", path + 'log.txt')
        
        while True:
            if (datetime.now()-now).seconds > params['min_wait']: break
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
            sleep(0.5)
            
        if params['HiPot']:
            HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vmax'], ptype=ptype, sat=True)
            n, Ispec = CalcIsSat(HP.SingleSave(f"{pathp}/IdVgsSat - {now}.csv", timeout=30), Elsa.GetT('t4k'))
            rename(f"{pathp}/IdVgsSat - {now}.csv", f"{pathp}/IdVgsSat - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
            WriteLog(f"n={format(n, '.3f')}, Ispec={format(Ispec, '.3e')} A", path + 'log.txt')
            sleep(60)
            
            if Ispec != 0:
                HP.SetVp(Ispec, params['Vmin'], params['Vmax'], 0.05, ptype=ptype)
                VTO=PlotVp(HP.SingleSave(f"{pathp}/VpVg - {now}.csv", timeout=30))
                rename(f"{pathp}/VpVg - {now}.csv", f"{pathp}/VpVg - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
                WriteLog(f"VTO={VTO} V", path + 'log.txt')
                sleep(60)
                
            HP.SetVds(params['Vmin'], params['Vmax'], params['Vstep'], params['Vgmin'], params['Vgmax'], params['Vgstep'], ptype=ptype)
            Plot(HP.SingleSave(f"{pathp}/IdVds - {now}.csv", timeout=30), 'Vd', 'Id')
            rename(f"{pathp}/IdVds - {now}.csv", f"{pathp}/IdVds - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
            sleep(60)
            
        print(INO.opench(0))
        WriteLog('', path + 'log.txt')
        return 0
    #
    #
    #######################
        
    ####################### Measure CrossBridge
    #
    #
    if 'c' in device.lower():

        HP.Set2P(params['Ifmin']/params['Iffactor'], params['Ifmax']/params['Iffactor'], params['Ifpoints'], SMUN='SMU1', SMUP='SMU2', Comp=params['VComp'])
            
        now=datetime.now().strftime('%y%m%d %H%M')
        Rshort=Plot2P(HP.SingleSave(f"{pathp}/2P - {now}.csv", timeout=30))
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(f"{pathp}/2P - {now}.csv", f"{pathp}/2P - {temp} - {now}.csv")
        rename(f"{pathp}/2P - {now}.png", f"{pathp}/2P - {temp} - {now}.png")
            
        WriteLog(f"Rshort={format(Rshort, '.2f')}", path + 'log.txt')
        with open(f"{pathp}/RxT 2P.log", 'a') as RxT2p:
            RxT2p.write(f"{temp},{format(Rshort, '.2f')}\n")

    if 'cb' in device.lower():
    
        HP.Set4P(params['Ifmin'], params['Ifmax'], params['Ifpoints'], Im='SMU1', Ip='SMU2', Comp=params['VComp'])
        
        now=datetime.now().strftime('%y%m%d %H%M')
        RCB=Plot2P(HP.SingleSave(f"{pathp}/4P - {now}.csv", timeout=30))
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(f"{pathp}/4P - {now}.csv", f"{pathp}/4P - {temp} - {now}.csv")
        rename(f"{pathp}/4P - {now}.png", f"{pathp}/4P - {temp} - {now}.png")
        
        WriteLog(f"RCB={format(RCB, '.2e')}", path + 'log.txt')
        with open(f"{pathp}/RxT 4P.log", 'a') as RxT4p:
            RxT4p.write(f"{temp},{format(RCB, '.2e')}\n")

    now=datetime.now()
    while True:
        if (datetime.now()-now).seconds > params['min_wait']: break
        prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
        sleep(0.5)
            
    print(INO.opench(0))
    WriteLog('', path + 'log.txt')
    return 0
    #
    #
    #######################

In [10]:
params = {
'Vmin' : 0,
'Vmax' : 1.5,
'Vstep' : 0.02,

'Vgmin' : 0.6,
'Vgmax' : 1.4,
'Vgstep' : 0.2,

'Vfmin' : 0,
'Vfmax' : 1.5,
'Vfstep' : 0.01,
'IComp' : 1e-3,

'Ifmin' : -5e-3,
'Ifmax' : 5e-3,
'Ifpoints' : 50,
'Iffactor' : 50,
'VComp' : 1.5,

'Vd' : 0.025,
'temp' : 3.338,
'HiPot' : False,
'min_wait' : 30
}

DeviceList = ['TP2', 'CB1', 'TN12', 'TN4', 'DN1']
# DeviceList = ['', '', '', 'TN4', '', '']
path = "C:/Users/Zucchi/Documents/Medidas/241007 - TN4 TN7 TN12 TP11/Warmup/"
makedirs(path, exist_ok=True)

In [11]:
###### Cold measurement

prog_bar=display('',display_id=True)

start=datetime.now()
WriteLog(f"#  Measurement 0 - {Elsa.GetT('t4k')} - {start.strftime('%y%m%d %H%M')}\n", path+'log.txt')

for chn, device in enumerate(DeviceList):
    if device:
        TestDevice(device, chn, path, params)
WriteLog(f"Measurement 0 end - {Elsa.GetT('t4k')}. Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path + 'log.txt')

clear_output(wait=True)
print("Pre warmup measurement done")

Pre warmup measurement done


In [12]:
####### Wait PT off

while Elsa.GetPulseTubeStatus():
    sleep(30)
last_temp=4

In [14]:
###### Cooldown measurement

freq_temp=10
MsrNo=3 ## Take {MsrNo} measurements
MsrWait=1 ## wait {MsrWait} minutes

prog_bar=display('',display_id=True)
last_temp=4

while(last_temp < 280):
    current_temp=Elsa.GetT('t4k')
    prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")
    
    #loop until temp changes and is a multiple of freq_temp
    while last_temp==np.around(current_temp) or ((current_temp%freq_temp) > 0.5 and (current_temp%freq_temp) < (freq_temp-0.5)):
        for i in range(15):
            sleep(1)
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")
        current_temp=Elsa.GetT('t4k')
        
    last_temp=np.around(current_temp)
    start=datetime.now()
    
    for Msr in range(MsrNo):
        plt.close('all')
        clear_output(wait=True)
        prog_bar=display('',display_id=True)
        
        WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {datetime.now().strftime('%y%m%d %H%M')}\n", path+'log.txt')
        for chn, device in enumerate(DeviceList):
            if device:
                TestDevice(device, chn, path, params)
        WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path + 'log.txt')

        current_temp=Elsa.GetT('t4k')
        for i in range(int(MsrWait*60)):
            sleep(1)
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")
        clear_output(wait=True)
print("Warmup measurement done

'03:34:22 Waiting minimum 30s'

# 142.111 K - Measurement 3 - 241023 0330

Open INO: ch1

## Ch 1 TP2
Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Starting IdxVgs. Duration: 35 s                                    
Vth=-1.093 V
SecDer=-0.74 V
YFunc Fail
Closed

Open INO: ch2

## Ch 2 CB1
Set 2P - 12
I=(-0.0001, 0.0001), 50 Points
Starting 2P - 12. Duration: 7 s                                    
Rshort=30.35
Set 4P
I=(-0.005, 0.005), 50 Points
Starting 4P. Duration: 20 s                                    
RCB=3.34e-02
Closed

Open INO: ch3

## Ch 3 TN12
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 26 s                                    
Vth=3.7 V
SecDer=-1.67 V
YFunc Fail
Closed



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [17]:
HP.close()
INO.close()